## **1.2.4 Extractie Totaal attributen data uit vectorlaag**
Voor de data analyse van de waarnemingen per huizen binnen een bouwjaar cluster is gewerkt met het aandeel huizen binnen een bouwjaar cluster. Hiervoor is met dit script bepaalt hoeveel huizen in Nederland binnen ieder bouwjaar cluster vallen volgens het [Basisregistratie Adressen en Gebouwen (BAG)]( https://www.digitaleoverheid.nl/overzicht-van-alle-onderwerpen/stelsel-van-basisregistraties/10-basisregistraties/bag/).

**Dit script is doelmatig geschreven en mogelijk niet buiten het project/de dataset inzetbaar.**

In [1]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import sys
import os

De code is doelmatig geschreven en heeft hierdoor weinig flexebiliteit. Wel moet de locatie van de kaart opgegeven worden en is het mogelijk de range van de clusters te veranderen.

In [2]:
# Geef hier de locatie van de BAG.gdb
Kaart = Path(r"Input_data/basisregistratie_adressen_en_gebouwen_(BAG)_05-10-2021/BAG.gdb")

# Naam van de Kaartlaag binnen het BAG.gdb waar de bouwjaren in benoemd staan
Kaartlaag = "Pand"

# Kolom naam met bouwjaren
KolomNaam = "bouwjaar"

# Geef hier de ranges van de bouwjaar clusters
Clusters = [[1000, 1600], [1601, 1800], [1801, 1900], [1901, 1940], [1941, 1980], [1981, 2005]]

## Het script

In [3]:
# Lees het kaart-bestand in. Als het niet van het goede bestandstype is word het script onderbroken.
if 'KaartGDF' not in globals():
    if Kaart.suffix == ".gdb":
        print(f"{Kaart.stem + Kaart.suffix} word ingelanden. afhankelijk van de bestandsgrote kan dit enkele minuten duren.")
        KaartGDF = gpd.read_file(Kaart, layer=Kaartlaag, engine="pyogrio", use_arrow=True)
    else:
        sys.exit(f"Script gestopt. Het kaart bestand is {Kaart.suffix}, dit moet type .gdb zijn")

print("Berekenen...")

# Maakt een nieuwe kolom in de KaartGDF van de kaart en classificeert voor iedere regel in welk cluster deze past
KaartGDF['Cluster'] = pd.cut(KaartGDF[KolomNaam], bins=[cls[0] - 1 for cls in Clusters] + [Clusters[-1][1]], labels=[f'{cls[0]}-{cls[1]}' for cls in Clusters])

# Telt het aantal regels wa aan ieder cluster is toegewezen en slaat de uitkomsten op in "OutputDF"
HuizenPerCluster = KaartGDF['Cluster'].value_counts().sort_index()
OutputDF = pd.DataFrame({'Clusters': HuizenPerCluster.index, 'Aantal': HuizenPerCluster.values})

# Maakt een kolom aan in "OutputDF" met waardes 1 tot 6 voor gemakkelijkere verwijzingen in latere analyses
OutputDF['Jaargroep code'] = range(1, len(OutputDF) + 1)

# Veranderd de volgorde van de kolommen voor overzicht
OutputDF = OutputDF[['Jaargroep code', 'Clusters', 'Aantal']]

# Slaat de output op in een excel bestand
print("Opslaan...")
if not os.path.exists("Output"):
    os.makedirs("Output")
excel_file = rf"Output/Totaal bouwjaar {Kaart.stem}.xlsx"
OutputDF.to_excel(excel_file, index=False, engine='openpyxl')
print(f"Output locatie: {excel_file}")


BAG.gdb word ingelanden. afhankelijk van de bestandsgrote kan dit enkele minuten duren.
Berekenen...
Opslaan...
Output locatie: Output/Totaal bouwjaar BAG.xlsx
